In [5]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=03a99499400d64cafc9e6f9e80b3984f5606b832d563b83778b69d3c63b211e2
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [6]:
from pyspark import SparkConf, SparkContext

In [7]:
conf = SparkConf().setMaster('local').setAppName('movie')

In [8]:
spark = SparkContext.getOrCreate(conf=conf)

## <span style="line-height:1.7">**u.data**와 **u.item**을 <b><font color='red'>RDD</font></b>로 로드하여 <font color='red'>**101명 이상**의 user</font>가 평가한 영화 중 <b><font color='blue'>평균 평점이 가장 높은 영화의 제목(title)</font></b>을 찾는 과정을 보이시오.</span>

### **u.data**
- **\t**으로 구분
- <font color='blue'>**user ID**, **movie ID**, **ratings**, **time stamp**</font>
  > - <b>user ID, movie ID</b>: user 및 영화의 ID
  > - <b>ratings</b>: user의 movie 평점
  > - <b>time stamp</b>: 평졈을 입력한 시간

### **u.item**
- **ㅣ**(verticla bar, or)으로 구분
- <font color='blue'>**movie ID**, **movie title**, **release data**, **video release data**, **imdb url**, **genre**</font>
  > - <b>movie ID</b>: 영화 ID
  > - <b>movie title</b>: 영화 제목 (string)
  > - <b>release data</b>: 영화 개봉일
  > - <b>video release data</b>: 비디오(dvd) 출시일
  > - <b>imdb url</b>: imdb 웹페이지 링크
  > - <b>나머지 column</b>: 장르

In [10]:
u_data_rdd = spark.textFile('u.data')
u_item_rdd = spark.textFile('u.item')

In [11]:
u_data_rdd = u_data_rdd.map(lambda line: line.split('\t')) \
  .map(lambda x: (int(x[0]), int(x[1]), float(x[2])))

u_item_rdd = u_item_rdd.map(lambda line: line.split('|')) \
  .map(lambda x: (int(x[0]), x[1]))

In [12]:
joined_rdd = u_data_rdd.join(u_item_rdd)

In [13]:
filtered_rdd = joined_rdd.filter(lambda x: x[1][0] >= 101)

In [14]:
average_ratings_rdd = filtered_rdd.map(lambda x: (x[1][1], x[0])) \
  .aggregateByKey((0, 0), lambda x, y: (x[0] + y, x[1] + 1), lambda x, y: (x[0] + y[0], x[1] + y[1])) \
  .map(lambda x: (x[0], x[1][0] / x[1][1]))

In [15]:
highest_rated_movie = average_ratings_rdd.sortBy(lambda x: x[1], ascending=False).first()

In [16]:
print(highest_rated_movie[0])

Killing Zoe (1994)
